In [ ]:
import torch
from tqdm import tqdm
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("pythia-70m-deduped", device="mps")
layer = 3

Loaded pretrained model pythia-70m-deduped into HookedTransformer


In [2]:
model.cfg.d_model**2

262144

In [3]:
_, cache = model.run_with_cache("Given the existence as uttered forth in the public works of Puncher and Wattmann of a personal God quaquaquaqua with white beard quaquaquaqua outside time without extension who from the heights of divine apathia divine athambia divine aphasia loves us dearly with some exceptions for reasons unknown but time will tell and suffers like the divine Miranda with those who for reasons unknown but time will tell are plunged in torment plunged in fire whose fire flames if that continues and who can doubt it will fire the firmament that is to say blast hell to heaven so blue still and calm so calm with a calm which even though intermittent is better than nothing but not so fast and considering what is more that as a result of the labors left unfinished crowned by the Acacacacademy of Anthropopopometry of Essy-in-Possy of")
cache['resid_pre', layer].shape, cache['mlp_out', layer].shape

(torch.Size([1, 176, 512]), torch.Size([1, 176, 512]))

In [4]:
def mlp(x):
    return model.blocks[layer].mlp(model.blocks[layer].ln2(x))

In [5]:
l0s = []
for pre, out in tqdm(zip(cache['resid_pre', layer][0], cache['mlp_out', layer][0])):
    assert torch.allclose(mlp(pre), out, atol=1e-4)
    jacobian = torch.autograd.functional.jacobian(mlp, pre)
    l0s.append((jacobian > 0).sum())
sum(l0s) / len(l0s)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
176it [00:54,  3.22it/s]


tensor(133108.1719, device='mps:0')